In [461]:
import pandas as pd
import numpy as np


In [140]:
train = pd.read_csv("train.csv")
testc = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

## Data Exploration

In [141]:
train.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [142]:
test.head()

,PID
0,1028890
1,1021443
2,1016194
3,1017298
4,1006098


In [143]:
sample.head()

,PID,Event1,Event2,Event3,Event4,Event5,Event6,Event7,Event8,Event9,Event10
0,1028890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1021443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1016194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
print train.shape,test.shape

(766787, 3) (3000, 1)


In [145]:
np.setdiff1d(train.PID,test.PID)

array([], dtype=int64)

In [146]:
len(train.Event.unique())

6472

In [147]:
len(train.PID.unique())

3000

In [148]:
(test.PID.value_counts()>1).describe()

count      3000
unique        1
top       False
freq       3000
Name: PID, dtype: object

In [149]:
(train.PID.value_counts()>50).describe()

count     3000
unique       2
top       True
freq      2891
Name: PID, dtype: object

In [150]:
train.loc[train['PID'] == 1029677].Event

751362    7288
751363    7245
751364    8102
751365    7102
751366    9928
751367    3470
751368    2649
751369    V723
751370    9939
751371    9900
751372    8100
751373    8389
751374    8390
751375    8391
Name: Event, dtype: object

In [151]:
event_counts = train.Event.value_counts()
len(event_counts[event_counts>40])

2316

In [152]:
event_counts_1011411 = train.loc[train['PID'] == 1011411].Event.value_counts()
event_counts_1011411

A465    91
4140    83
9921    80
5856    78
9099    76
2500    62
7865    50
7860    48
J164    45
4019    39
9301    39
2449    37
1890    33
Q408    33
9923    32
4280    32
7101    31
2809    30
1985    30
J127    30
5888    30
V451    29
6811    26
4039    25
5188    25
2852    25
J175    23
9929    23
9922    21
8734    21
        ..
1721     1
9399     1
6259     1
2688     1
2684     1
3532     1
7362     1
2395     1
9331     1
8369     1
7361     1
8373     1
8361     1
7728     1
8830     1
V151     1
7893     1
3197     1
566      1
3464     1
1918     1
J240     1
7355     1
1991     1
7356     1
7847     1
4442     1
V764     1
3747     1
3000     1
Name: Event, dtype: int64

### Hypothesis : Find pattern in a patients medical activity in some fixed interval like 6month or a year(2.76 year in this case) - score: 0.0722

In [153]:
### 1. Fixing jan 1, 2014 to check age of a record

In [154]:
from datetime import datetime

date_threshold = datetime.strptime("January, 2014","%B, %Y")


In [155]:
train["Date"] = pd.to_datetime(train["Date"],format="%Y%m")

train["record_age"] = train.Date.apply(lambda x: (date_threshold - x).days)

In [156]:
train.head()

,PID,Date,Event,record_age
0,1028890,2011-01-01,2186,1096
1,1028890,2011-01-01,7087,1096
2,1028890,2011-01-01,4848,1096
3,1028890,2011-01-01,2214,1096
4,1028890,2011-02-01,7087,1065


In [157]:
#Find Records of last x months and sort them by date
train_last_x_months = train.loc[train['record_age']<=1010]
train_last_x_months.sort_values(by="Date",ascending=False,inplace=True)

train_last_x_crosstab = pd.crosstab(index=train_last_x_months['PID'], columns=train_last_x_months['Event'])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [158]:
len(train_last_x_months.PID.unique())

3000

In [159]:
np.setdiff1d(test["PID"],train_last_x_months["PID"])

array([], dtype=int64)

In [160]:
# train_last_six_crosstab = pd.crosstab(index=train_last_six_months['PID'], columns=train_last_six_months['Event'])
submit3 = train_last_x_crosstab.loc[:,train_last_x_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [161]:
submit3.reset_index(drop=True)

,PID,Event1,Event2,Event3,Event4,Event5,Event6,Event7,Event8,Event9,Event10
0,1000001,3074,3263,2533,2178,2632,3041,3016,3272,9921,2582
1,1000011,9921,8470,3106,8100,9928,8502,9711,2335,7231,7890
2,1000019,7682,7681,9928,6558,7680,6522,V270,9397,9920,7194
3,1000025,3254,3641,2635,2674,3419,3410,8444,8443,5852,9921
4,1000029,2533,2773,3638,9921,4011,8830,7373,3616,V723,3051
5,1000043,3082,9080,3091,9921,3591,3092,3272,3665,3201,2843
6,1000068,2334,9711,9703,3263,9701,3478,7245,3454,8502,9928
7,1000075,1963,2777,3317,2809,2449,9921,4019,3641,5693,2859
8,1000079,4140,9928,7865,3320,V586,4019,9301,2387,2500,7860
9,1000082,3324,3272,2632,2627,3729,2978,V726,9939,G020,8690


In [162]:
submit3.to_csv("third_sub.csv",index=False)

### Hypothesis : Take 2013 details score : 0.08625

In [178]:
train_2013 = train.loc[train['record_age']<=365]

In [170]:
PID_left = np.setdiff1d(train.PID,train_2013.PID)
train_2013_left_ids = train.loc[train['PID'].isin(PID_left)]

In [173]:
train_latest = pd.concat([train_2013,train_2013_left_ids],axis=0)

In [174]:
len(train_latest.PID.unique())

3000

In [175]:
train_latest_crosstab = pd.crosstab(index=train_latest['PID'],columns=train_latest['Event'])


In [176]:
submit4 = train_latest_crosstab.loc[:,train_latest_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [177]:
submit4.to_csv('fourth_submit.csv',index=False)

### Hypothesis : Take last six months the score : 0.08897 add filtering for missing ids to be within last 2.57 years score becomes 0.08979 , change six months threshold to 250 days gives score of 0.09321

In [453]:
train_six_months = train.loc[train['record_age']<=300]

In [454]:
PID_left_six = np.setdiff1d(train.PID,train_six_months.PID)
train_six_left_ids = train.loc[(train['PID'].isin(PID_left_six)) & (train['record_age']<=1010)]

In [455]:
train_six = pd.concat([train_six_months,train_six_left_ids],axis=0)

In [456]:
len(train_six.PID.unique())

3000

In [457]:
train_six_crosstab = pd.crosstab(index=train_six['PID'],columns=train_six['Event'])


In [458]:
submit5 = train_six_crosstab.loc[:,train_six_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [459]:
submit5.to_csv('twelve_submit.csv',index=False)

### Hypothesis : Last six months in original order score: 0.00025

In [210]:
train_six_months_order = train.loc[train['record_age']<=180]
PID_left_six_order = np.setdiff1d(train.PID,train_six_months_order.PID)
train_six_order_left_ids = train.loc[train['PID'].isin(PID_left_six)]
train_six_order = pd.concat([train_six_months_order,train_six_order_left_ids],axis=0)

In [211]:
len(train_six_order.PID.unique())

3000

In [212]:
train_six_order_crosstab = pd.crosstab(index=train_six_order['PID'],columns=train_six_order['Event'])


In [217]:
submit6 = train_six_order_crosstab.loc[:,train_six_order_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.iloc[-10:].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [218]:
submit6.to_csv('six_submit.csv',index=False)

### Hypothesis: Last two months score : 0.07066 Not Able to Catch the pattern properly

In [200]:
train_two_months = train.loc[train['record_age']<=60]

In [201]:
len(train_two_months.PID.unique())

2102

In [202]:
pid_left_two_months = np.setdiff1d(train.PID,train_two_months.PID)

In [204]:
train_two_months_left = train.loc[train['PID'].isin(pid_left_two_months)]

In [205]:
train_two = pd.concat([train_two_months,train_two_months_left],axis=0)

In [206]:
len(train_two.PID.unique())

3000

In [207]:
train_two_crosstab = pd.crosstab(index=train_two['PID'],columns=train_two['Event'])


In [208]:
submit7 = train_two_crosstab.loc[:,train_two_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [209]:
submit7.to_csv('seven_submit.csv',index=False)

### Hypothesis Multiple Rules based on Record availability for time periods : 6months,23 months and 33 months score 0.08919 probably what Pavleen Tried

In [407]:
train_6 = train.loc[train['record_age']<=180]

In [408]:
missing_ids_6 = np.setdiff1d(train.PID,train_6.PID)
print len(missing_ids_6)
missing_values = train[train.PID.isin(missing_ids_6)]

185


In [409]:
train_23 = train.loc[(train['record_age']<=700)&train['PID'].isin(missing_ids_6)]

In [410]:
missing_ids_23 = np.setdiff1d(missing_ids_6,train_23.PID)

In [411]:
len(missing_ids_23)

3

In [412]:
train_27 = train.loc[(train['record_age']<=1010)&train['PID'].isin(missing_ids_23)]

In [413]:
train_multi_rule = pd.concat([train_6,train_23,train_27],axis=0)

In [414]:
len(train_multi_rule.PID.unique())

3000

In [415]:
train_multi_crosstab = pd.crosstab(index=train_multi_rule['PID'], columns=train_multi_rule.Event)

In [416]:
submit8 = train_multi_crosstab.loc[:,train_multi_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [417]:
submit8.to_csv("eight_sub.csv",index=False)

### Hypothesis increase to ten months and for others same approach score 0.08863

In [285]:
train_10 = train.loc[train['record_age']<=300]

In [286]:
missing_ids_10 = np.setdiff1d(train.PID,train_x.PID)

In [287]:
train_missing_10 = train.loc[train['PID'].isin(missing_ids_10)]

In [288]:
train_10_final = pd.concat([train_10,train_missing_10],axis=0)

In [289]:
len(train_10_final.PID.unique())

3000

In [290]:
train_10_crosstab = pd.crosstab(index=train_10_final.PID, columns=train_10_final.Event)

In [291]:
submit9 = train_10_crosstab.loc[:,train_10_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [292]:
submit9.to_csv('nine_sub.csv',index=False)

### Hypothesis : Take every user's last 50 records for predicting score : 0.06904

In [336]:
record_count = train.PID.value_counts()

In [342]:
train.PID.value_counts().describe()

count    3000.000000
mean      255.595667
std       237.081149
min        11.000000
25%       119.000000
50%       182.000000
75%       312.000000
max      2476.000000
Name: PID, dtype: float64

In [338]:
train.PID.value_counts().mode()

0    121
1    131
dtype: int64

In [357]:
train['record_count'] = train.PID.apply(lambda x: record_count[x])

In [358]:
len(train.loc[train['record_count']<50].PID.unique())

106

In [402]:
train_by_pid = train.groupby('PID')
train_last_50 = train_by_pid.tail(500)

In [403]:
len(train_last_50.PID.unique())

3000

In [404]:
train_group_crosstab = pd.crosstab(index=train_last_50['PID'],columns=train_last_50['Event'])

In [405]:
submit10 = train_group_crosstab.loc[:,train_group_crosstab.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()

In [406]:
submit10.to_csv('submit10.csv',index=False)